In [46]:
import ete3
import re
import pickle as pkl
import collections
import subprocess

%cd /Sites/reconcile/

/Sites/reconcile


In [2]:
%ls

example/      ranger.input


In [22]:
ranger  = open('example/ranger.output.1').readlines()
species = ete3.Tree(ranger[4], format=1)
gene    = ete3.Tree(ranger[7], format=1)
gene_brLen = ete3.Tree('/Users/thiberio/Downloads/haitao/gene.tre')

In [18]:
transfers    = re.findall('^.*: Transfer,.*$', '\n'.join(ranger), re.M)
duplications = re.findall('^.*: Duplication,.*$', '\n'.join(ranger), re.M)

In [49]:
pair = []
for t in transfers:
    donor, recipient = re.search('Transfer, Mapping --> (\S+), Recipient --> (\S+)', t).groups()
    pair.append(frozenset([donor, recipient]))

In [50]:
collections.Counter(pair)

Counter({frozenset({'SAR202clusterbacteriumIo17ChloroG1',
                    'SAR202clusterbacteriumIo17ChloroG3'}): 1,
         frozenset({'SAR202clusterbacteriumCaspChloroG2',
                    'SAR202clusterbacteriumMPSIndSRR3963457G1'}): 1,
         frozenset({'SAR202clusterbacteriumAe2ChloroG2', 'n3'}): 1,
         frozenset({'SAR202clusterbacteriumIo17ChloroG6', 'n10'}): 1,
         frozenset({'SAR202clusterbacteriumCaspChloroG3', 'n8'}): 1,
         frozenset({'SAR202clusterbacteriumCaspChloroG2',
                    'SAR202clusterbacteriumIo17ChloroG6'}): 1,
         frozenset({'SAR202clusterbacteriumIo17ChloroG5',
                    'SAR202clusterbacteriumMPSIndSRR3963457G1'}): 1,
         frozenset({'SAR202clusterbacteriumCaspChloroG2',
                    'SAR202clusterbacteriumIo17ChloroG5'}): 1,
         frozenset({'SAR202clusterbacteriumIo17ChloroG9', 'n10'}): 1,
         frozenset({'n6', 'n7'}): 1,
         frozenset({'SAR202clusterbacteriumCaspChloroG3', 'n5'}): 1})

In [28]:
for t in transfers:
    node_name, descendant1, descendant2 = re.search('^(\S+) = LCA\[(\S+), (\S+)\]', t, re.M).groups()
    equivalent_node = gene_brLen.get_common_ancestor(descendant1, descendant2)
    equivalent_node.add_feature('rangerID', node_name)

In [37]:
out = open('yeah.pkl', 'wb')
pkl.dump(gene_brLen, out)
out.close()

In [38]:
for loop in range(10):
    subprocess.call(['/work/ranger/CorePrograms/Ranger-DTL.mac',
                     '-i',
                     'ranger.input',
                     '-o',
                     'reconciliations/ranger.output.%i' % (loop+1), 
                     '--seed',
                     str(loop+1)
                    ])
aggregate = subprocess.getoutput('/work/ranger/CorePrograms/AggregateRanger.mac reconciliations/ranger.output.')
out = open('aggregated.ranger', 'w')
out.write(aggregate)
out.close()

Tree node '' (-0x7fffffffee373ed9)